In [15]:
%autosave 0
%matplotlib inline

Autosave disabled


In [16]:
import codecs
import pymarc
import sqlite3
import yaml
from collections import defaultdict
import random

In [17]:
INPUT_MARCXML_FILE = "/src/data/input/metadata/marc21.xml"
MARCXML_PARSER_CONF = "/src/config/parsing_conf.yaml"
INPUT_SAMPLE_SIZE = 10

### Task 1 - Count number of records

In [18]:
#Done - in /src/input/metadata/marcxml_parsing

### Task 2 - Tally frequency of tags

In [19]:
#Done - in /src/input/metadata/marcxml_parsing

### Task 3 - Parse marcxml

In [20]:
marc_records = pymarc.parse_xml_to_array(INPUT_MARCXML_FILE)
try: marc_records = random.sample(marc_records, INPUT_SAMPLE_SIZE)
except: pass

### Task 4 - Map records to dictionary

In [21]:
def load_parser_conf(conf_file):
    with open(conf_file, "r") as stream:
        conf = yaml.load(stream)
    return conf

def fix_uid_mapping(marc_field):
    if 'u' in marc_field:
        if 'z' in marc_field: 
            uid = marc_field["z"].split(":")[0]
            if uid is not None:
                marc_field["u"] = uid + ": "+ marc_field["u"]
        else: marc_field.delete_subfield("u")
    return marc_field

def map_marcxml_record(marc_record, conf):
    marc_dict = defaultdict(set)
    for entry in conf:
        (k, v), = entry.items()
        field, sub_field = k.split("$")
        for marc_field in marc_record.get_fields(field):
            if field == "856": marc_field = fix_uid_mapping(marc_field)
            marc_subfield = marc_field[sub_field]
            if marc_subfield is not None: 
                marc_dict[v].add(marc_subfield)
    return dict(marc_dict)

def map_marcxml_records(marc_records, conf_file):
    conf = load_parser_conf(conf_file)
    marc_records_tf = map(lambda record: map_marcxml_record(record, conf), marc_records)
    return list(marc_records_tf)

marc_records = map_marcxml_records(marc_records, MARCXML_PARSER_CONF)

In [33]:
marc_records[9]

{'collection_id': {'.b34478358'},
 'date_created': {'ca. 1925-1929.'},
 'language_code': {'eng '},
 'location_code': {'u-at-we'},
 'location_division': {'Western Australia'},
 'location_name': {'Cottesloe Beach (W.A.)',
  'Forrest Place (Perth, W.A.)',
  'Perth (W.A.)',
  'St Georges Terrace (Perth, W.A.)'},
 'note_general': {'Title devised by cataloguer.'},
 'note_public': {'221711PD: Japanese sailors on the Swan River during a visit by the Japanese Navy in June 1928',
  '221712PD: Japanese sailors on the Swan River during a visit by the Japanese Navy in June 1928',
  '221713PD: Japanese sailors marching during a visit to Perth in June 1928',
  '221714PD: Swimming in the surf at Cottesloe Beach, 1925-1929',
  '221715PD: Cottesloe Beach, 1925-1929',
  '221716PD: Cottesloe Beach, 1925-1929',
  '221717PD: Forrest Place, 1925-1929',
  '221718PD: Murray St and Hay St, Perth showing Grand Theatre, Aherns and the Town Hall, 1925-1929',
  '221719PD: View over Perth, 1925-1929',
  "221720PD: S

In [12]:
def reverse_nested_dict(original_dict):
    for note_url in original_dict["note_url"]:
        image_id = note_url.split("/")[0]

list(map(lambda record: reverse_nested_dict(record), marc_records))

AttributeError: 'set' object has no attribute 'items'